In [ ]:
# Import necessary libraries
from google.colab import drive
import scipy.io
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Create the destination folder if it doesn't exist
output_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Path to your .mat files
mat_files_dir = '/content/drive/My Drive/Colab Notebooks/thermal cfrp data'

def save_3d_array_as_images(array, output_folder, prefix):
    """
    Save each 2D slice of a 3D array as an image
    array: 3D numpy array
    output_folder: destination folder path
    prefix: prefix for the image filenames
    """
    # Get the number of slices
    num_images = array.shape[0]

    for i in range(num_images):
        # Extract the 2D slice
        img_data = array[i, :, :]

        # Normalize to 0-255 range for PNG format
        img_data = ((img_data - img_data.min()) * (255.0 / (img_data.max() - img_data.min()))).astype(np.uint8)

        # Create image from array
        img = Image.fromarray(img_data)

        # Save the image
        img_path = os.path.join(output_folder, f'{prefix}_image_{i:04d}.png')
        img.save(img_path)

        # Print progress every 10 images
        if (i + 1) % 10 == 0:
            print(f'Saved {i + 1}/{num_images} images from {prefix}')

# Process each .mat file
mat_files = ['P1X1_train_norm.mat', 'P1X2_train_norm.mat',
             'P2X1_train_norm.mat', 'P2X2_train_norm.mat']

for filename in mat_files:
    mat_path = os.path.join(mat_files_dir, filename)
    if os.path.exists(mat_path):
        print(f'\nProcessing {filename}...')
        # Load the .mat file
        mat_data = scipy.io.loadmat(mat_path)

        # Print the keys in the .mat file
        print(f'Keys in {filename}:', mat_data.keys())

        # Find the main data array (excluding metadata keys that start with '__')
        data_keys = [key for key in mat_data.keys() if not key.startswith('__')]

        if data_keys:
            for key in data_keys:
                data_array = mat_data[key]
                print(f'Shape of array in {key}:', data_array.shape)

                # Save images
                prefix = filename.split('.')[0]
                save_3d_array_as_images(data_array, output_dir, prefix)
        else:
            print(f'No data arrays found in {filename}')
    else:
        print(f'File not found: {filename}')

print('\nImage extraction complete')

# Print total number of saved images
total_images = len([name for name in os.listdir(output_dir) if name.endswith('.png')])
print(f'\nTotal images saved: {total_images}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processing P1X1_train_norm.mat...
Keys in P1X1_train_norm.mat: dict_keys(['__header__', '__version__', '__globals__', 'P1X1_train_norm'])
Shape of array in P1X1_train_norm: (120, 250, 200)
Saved 10/120 images from P1X1_train_norm
Saved 20/120 images from P1X1_train_norm
Saved 30/120 images from P1X1_train_norm
Saved 40/120 images from P1X1_train_norm
Saved 50/120 images from P1X1_train_norm
Saved 60/120 images from P1X1_train_norm
Saved 70/120 images from P1X1_train_norm
Saved 80/120 images from P1X1_train_norm
Saved 90/120 images from P1X1_train_norm
Saved 100/120 images from P1X1_train_norm
Saved 110/120 images from P1X1_train_norm
Saved 120/120 images from P1X1_train_norm

Processing P1X2_train_norm.mat...
Keys in P1X2_train_norm.mat: dict_keys(['__header__', '__version__', '__globals__', 'P1X2_train_norm'])
Shape of array in P1X2_train_norm: (120, 250, 2

In [ ]:
import numpy as np
import cv2
from google.colab import drive
import os
from PIL import Image

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define input and output directories
input_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
output_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset gradientSubtraction'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def apply_gradient_subtraction(image):
    # Convert to float32 for calculations
    img_float = image.astype(np.float32)

    # Calculate gradients using Sobel operators
    grad_x = cv2.Sobel(img_float, cv2.CV_32F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img_float, cv2.CV_32F, 0, 1, ksize=3)

    # Calculate gradient magnitude
    gradient = np.sqrt(grad_x**2 + grad_y**2)

    # Subtract gradient from original image
    result = img_float - gradient

    # Normalize result to 0-255 range
    result = ((result - result.min()) * (255.0 / (result.max() - result.min()))).astype(np.uint8)

    return result

# Process all images
for filename in os.listdir(input_dir):
    if filename.endswith('.png'):
        # Read image
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply gradient subtraction
        processed_img = apply_gradient_subtraction(img)

        # Save processed image
        output_path = os.path.join(output_dir, f'grad_{filename}')
        cv2.imwrite(output_path, processed_img)

print("Gradient subtraction completed")

# Count processed images
total_processed = len([name for name in os.listdir(output_dir) if name.endswith('.png')])
print(f'Total images processed: {total_processed}')

Mounted at /content/drive
Gradient subtraction completed
Total images processed: 480


In [ ]:
!pip install fpdf2

import numpy as np
import cv2
import os
from PIL import Image
from fpdf import FPDF
import matplotlib.pyplot as plt
from datetime import datetime

def calculate_snr(img):
    """Calculate Signal-to-Noise Ratio"""
    img_float = img.astype(float)

    # Define ROI for signal
    threshold = np.mean(img_float) + np.std(img_float)
    signal_mask = img_float > threshold
    noise_mask = ~signal_mask

    if np.sum(signal_mask) > 0 and np.sum(noise_mask) > 0:
        signal_mean = np.mean(img_float[signal_mask])
        noise_std = np.std(img_float[noise_mask])
        snr = 20 * np.log10(signal_mean / noise_std)
    else:
        snr = 0

    return snr

def create_comparison_plot(original_img, processed_img, snr_orig, snr_proc, filename):
    """Create a comparison plot with SNR values"""
    plt.figure(figsize=(12, 5))

    # Plot original image
    plt.subplot(121)
    plt.imshow(original_img, cmap='gray')
    plt.title(f'Original (SNR: {snr_orig:.2f} dB)')
    plt.axis('off')

    # Plot processed image
    plt.subplot(122)
    plt.imshow(processed_img, cmap='gray')
    plt.title(f'Gradient Subtracted (SNR: {snr_proc:.2f} dB)')
    plt.axis('off')

    # Save plot
    plt.savefig(f'temp_{filename}.png', bbox_inches='tight', dpi=300)
    plt.close()

# Define directories
original_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
processed_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset gradientSubtraction'
output_dir = '/content/drive/My Drive/Colab Notebooks/analysis_results'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Add title page
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Thermal Image SNR Analysis Report', ln=True, align='C')
pdf.set_font('Arial', '', 12)
pdf.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', ln=True, align='C')

# Initialize lists to store metrics
all_metrics = []
filenames = []

# Process all images
print("Processing images...")
for filename in sorted(os.listdir(original_dir)):
    if filename.endswith('.png'):
        # Read images
        original_path = os.path.join(original_dir, filename)
        processed_path = os.path.join(processed_dir, f'grad_{filename}')

        if os.path.exists(processed_path):
            original_img = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
            processed_img = cv2.imread(processed_path, cv2.IMREAD_GRAYSCALE)

            # Calculate SNR
            snr_original = calculate_snr(original_img)
            snr_processed = calculate_snr(processed_img)

            # Store metrics
            all_metrics.append({
                'filename': filename,
                'SNR_original': snr_original,
                'SNR_processed': snr_processed,
                'SNR_improvement': snr_processed - snr_original
            })

            # Create comparison plot
            create_comparison_plot(original_img, processed_img, snr_original, snr_processed, filename)

            # Add to PDF
            pdf.add_page()
            pdf.set_font('Arial', 'B', 12)
            pdf.cell(0, 10, f'Image: {filename}', ln=True)
            pdf.image(f'temp_{filename}.png', x=10, w=190)

            # Add SNR values
            pdf.set_font('Arial', '', 10)
            pdf.cell(0, 10, f'SNR Improvement: {snr_processed - snr_original:.2f} dB', ln=True)

            # Clean up temporary files
            os.remove(f'temp_{filename}.png')

            # Print progress
            print(f"Processed {filename}")

# Add summary statistics
pdf.add_page()
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, 'Summary Statistics', ln=True)
pdf.set_font('Arial', '', 12)

# Calculate summary statistics
avg_snr_improvement = np.mean([m['SNR_improvement'] for m in all_metrics])
max_snr_improvement = np.max([m['SNR_improvement'] for m in all_metrics])
min_snr_improvement = np.min([m['SNR_improvement'] for m in all_metrics])

# Add statistics to PDF
pdf.cell(0, 10, f'Average SNR Improvement: {avg_snr_improvement:.2f} dB', ln=True)
pdf.cell(0, 10, f'Maximum SNR Improvement: {max_snr_improvement:.2f} dB', ln=True)
pdf.cell(0, 10, f'Minimum SNR Improvement: {min_snr_improvement:.2f} dB', ln=True)

# Create SNR improvement distribution plot
plt.figure(figsize=(10, 6))
improvements = [m['SNR_improvement'] for m in all_metrics]
plt.hist(improvements, bins=30, edgecolor='black')
plt.title('Distribution of SNR Improvements')
plt.xlabel('SNR Improvement (dB)')
plt.ylabel('Number of Images')
plt.savefig('snr_distribution.png', bbox_inches='tight', dpi=300)
plt.close()

# Add distribution plot to PDF
pdf.add_page()
pdf.image('snr_distribution.png', x=10, w=190)
os.remove('snr_distribution.png')

# Save PDF
pdf_path = os.path.join(output_dir, 'thermal_image_snr_analysis.pdf')
pdf.output(pdf_path)

print(f"\nAnalysis completed. PDF report saved to: {pdf_path}")

# Print overall statistics
print(f"\nSummary Statistics:")
print(f"Average SNR Improvement: {avg_snr_improvement:.2f} dB")
print(f"Maximum SNR Improvement: {max_snr_improvement:.2f} dB")
print(f"Minimum SNR Improvement: {min_snr_improvement:.2f} dB")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 9.2 MB/s eta 0:00:00


<ipython-input-4-4c44c1853ed1>:63: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
<ipython-input-4-4c44c1853ed1>:64: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, 'Thermal Image SNR Analysis Report', ln=True, align='C')
<ipython-input-4-4c44c1853ed1>:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-4-4c44c1853ed1>:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', ln=True, align='C')


Processing images...


<ipython-input-4-4c44c1853ed1>:101: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-4-4c44c1853ed1>:102: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Image: {filename}', ln=True)
<ipython-input-4-4c44c1853ed1>:106: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
<ipython-input-4-4c44c1853ed1>:107: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'SNR Improvement: {snr_processed - snr_original:.2f} dB', ln=True)


Processed P1X1_train_norm_image_0000.png
Processed P1X1_train_norm_image_0001.png
Processed P1X1_train_norm_image_0002.png
Processed P1X1_train_norm_image_0003.png
Processed P1X1_train_norm_image_0004.png
Processed P1X1_train_norm_image_0005.png
Processed P1X1_train_norm_image_0006.png
Processed P1X1_train_norm_image_0007.png
Processed P1X1_train_norm_image_0008.png
Processed P1X1_train_norm_image_0009.png
Processed P1X1_train_norm_image_0010.png
Processed P1X1_train_norm_image_0011.png
Processed P1X1_train_norm_image_0012.png
Processed P1X1_train_norm_image_0013.png
Processed P1X1_train_norm_image_0014.png
Processed P1X1_train_norm_image_0015.png
Processed P1X1_train_norm_image_0016.png
Processed P1X1_train_norm_image_0017.png
Processed P1X1_train_norm_image_0018.png
Processed P1X1_train_norm_image_0019.png
Processed P1X1_train_norm_image_0020.png
Processed P1X1_train_norm_image_0021.png
Processed P1X1_train_norm_image_0022.png
Processed P1X1_train_norm_image_0023.png
Processed P1X1_t

<ipython-input-4-4c44c1853ed1>:117: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 14)
<ipython-input-4-4c44c1853ed1>:118: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, 'Summary Statistics', ln=True)
<ipython-input-4-4c44c1853ed1>:119: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-4-4c44c1853ed1>:127: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Average SNR Improvement: {avg_snr_improvement:.2f} dB', ln=True)
<ipython-input-4-4c44c1853ed1>:128: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new


Analysis completed. PDF report saved to: /content/drive/My Drive/Colab Notebooks/analysis_results/thermal_image_snr_analysis.pdf

Summary Statistics:
Average SNR Improvement: 1.67 dB
Maximum SNR Improvement: 7.12 dB
Minimum SNR Improvement: -21.62 dB


In [ ]:
#------------------------ Training and Testing -----------------------------

In [ ]:
# 1. Clone YOLOv5
!git clone https://github.com/ultralytics/yolov5

# 2. Change directory to yolov5
%cd yolov5

# 3. Install requirements
!pip install -r requirements.txt

# 4. Start training
!python train.py \
--img 640 \
--batch 8 \
--epochs 50 \
--patience 10 \
--data "/content/drive/My Drive/Colab Notebooks/CFRP Defect Detection.v1i.yolov5pytorch/data.yaml" \
--weights yolov5m.pt \
--cache

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-02-14 18:19:00.094643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739557140.115288   26208 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739557140.121843   26208 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: niranjan-sh18 (niranjan-sh18-northumbria-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, 

In [ ]:
# YOLOv5 directory
%cd /content/yolov5

# Run testing on test dataset
!python val.py \
  --weights runs/train/exp/weights/best.pt \
  --data "/content/drive/My Drive/Colab Notebooks/CFRP Defect Detection.v1i.yolov5pytorch/data.yaml" \
  --img 640 \
  --task test

/content/yolov5
val: data=/content/drive/My Drive/Colab Notebooks/CFRP Defect Detection.v1i.yolov5pytorch/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
test: Scanning /content/drive/My Drive/Colab Notebooks/CFRP Defect Detection.v1i.yolov5pytorch/test/labels.cache... 48 images, 2 backgrounds, 0 corrupt: 100% 48/48 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 2/2 [00:02<00:00,  1.29s/it]
                   all         48        216      0.932      0.958 

In [ ]:
!zip -r yolov5.zip /content/yolov5

  adding: content/yolov5/ (stored 0%)
  adding: content/yolov5/.dockerignore (deflated 56%)
  adding: content/yolov5/.gitignore (deflated 55%)
  adding: content/yolov5/pyproject.toml (deflated 59%)
  adding: content/yolov5/__pycache__/ (stored 0%)
  adding: content/yolov5/__pycache__/hubconf.cpython-311.pyc (deflated 74%)
  adding: content/yolov5/__pycache__/export.cpython-311.pyc (deflated 60%)
  adding: content/yolov5/__pycache__/val.cpython-311.pyc (deflated 56%)
  adding: content/yolov5/README.zh-CN.md (deflated 76%)
  adding: content/yolov5/yolov5/ (stored 0%)
  adding: content/yolov5/yolov5/.dockerignore (deflated 56%)
  adding: content/yolov5/yolov5/.gitignore (deflated 55%)
  adding: content/yolov5/yolov5/pyproject.toml (deflated 59%)
  adding: content/yolov5/yolov5/__pycache__/ (stored 0%)
  adding: content/yolov5/yolov5/__pycache__/val.cpython-311.pyc (deflated 56%)
  adding: content/yolov5/yolov5/README.zh-CN.md (deflated 76%)
  adding: content/yolov5/yolov5/yolov5/ (stored 

In [ ]:
!mv yolov5.zip /content/drive/MyDrive/